# W04 Cassandra Python Tutorial

### Import packages

In [1]:
import pandas as pd
from cassandra.cluster import Cluster

### Create a session connection to Cassandra cluster

In [2]:
clstr = Cluster()
session = clstr.connect()

### Use session to 'talk' to cassandra

In [3]:
session.execute("DROP KEYSPACE IF EXISTS w04python")

In [4]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema


In [5]:
session.execute("CREATE KEYSPACE IF NOT EXISTS w04python WITH REPLICATION = {'class':'SimpleStrategy', 'replication_factor':2}")

In [6]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema
w04python


Create a student table for testing

In [7]:
session.execute("""
CREATE TABLE IF NOT EXISTS w04python.student ( 
    student_id INT, 
    f_name TEXT, 
    l_name TEXT, 
    course TEXT,
    grade FLOAT, 
    PRIMARY KEY(course, grade, student_id)
);
""")

### Load data

In [8]:
df = pd.read_csv('../w04.csv')
df

,student_id,f_name,l_name,course,grade
0,1000,'Tim','Smith','ISM6562',95
1,1001,'John','Jones','ISM6562',97
2,1002,'Jane','Williams','ISM6562',89
3,1003,'Betty','Johnson','ISM6564',78
4,1004,'Jim','Bean','ISM6564',65
5,1005,'Frank','Gambali','ISM6564',92
6,1006,'Judy','Garland','ISM6562',87


In [9]:
for index, row in df.iterrows():
    print(f"student_id = {row.student_id}, f_name ={row.f_name}, l_name = {row.l_name}, course = {row.course}, grade = {row.grade}")

student_id = 1000, f_name ='Tim', l_name = 'Smith', course = 'ISM6562', grade = 95
student_id = 1001, f_name ='John', l_name = 'Jones', course = 'ISM6562', grade = 97
student_id = 1002, f_name ='Jane', l_name = 'Williams', course = 'ISM6562', grade = 89
student_id = 1003, f_name ='Betty', l_name = 'Johnson', course = 'ISM6564', grade = 78
student_id = 1004, f_name ='Jim', l_name = 'Bean', course = 'ISM6564', grade = 65
student_id = 1005, f_name ='Frank', l_name = 'Gambali', course = 'ISM6564', grade = 92
student_id = 1006, f_name ='Judy', l_name = 'Garland', course = 'ISM6562', grade = 87


In [10]:
for index, row in df.iterrows():
    print(f"""
        INSERT INTO w04python.student (student_id, f_name, l_name, course, grade)     
        VALUES ({row[0]}, {row[1]}, {row[2]}, {row[3]}, {row[4]});
        """
       )
    session.execute(f"""
        INSERT INTO w04python.student (student_id, f_name, l_name, course, grade)     
        VALUES ({row[0]}, {row[1]}, {row[2]}, {row[3]}, {row[4]});
        """
       )


        INSERT INTO w04python.student (student_id, f_name, l_name, course, grade)     
        VALUES (1000, 'Tim', 'Smith', 'ISM6562', 95);
        

        INSERT INTO w04python.student (student_id, f_name, l_name, course, grade)     
        VALUES (1001, 'John', 'Jones', 'ISM6562', 97);
        

        INSERT INTO w04python.student (student_id, f_name, l_name, course, grade)     
        VALUES (1002, 'Jane', 'Williams', 'ISM6562', 89);
        

        INSERT INTO w04python.student (student_id, f_name, l_name, course, grade)     
        VALUES (1003, 'Betty', 'Johnson', 'ISM6564', 78);
        



        INSERT INTO w04python.student (student_id, f_name, l_name, course, grade)     
        VALUES (1004, 'Jim', 'Bean', 'ISM6564', 65);
        

        INSERT INTO w04python.student (student_id, f_name, l_name, course, grade)     
        VALUES (1005, 'Frank', 'Gambali', 'ISM6564', 92);
        

        INSERT INTO w04python.student (student_id, f_name, l_name, course, grade)     
        VALUES (1006, 'Judy', 'Garland', 'ISM6562', 87);
        


/var/folders/fc/_rjkdv156tv7hwc972mv_lsm0000gq/T/ipykernel_34665/45361915.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  VALUES ({row[0]}, {row[1]}, {row[2]}, {row[3]}, {row[4]});
/var/folders/fc/_rjkdv156tv7hwc972mv_lsm0000gq/T/ipykernel_34665/45361915.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  VALUES ({row[0]}, {row[1]}, {row[2]}, {row[3]}, {row[4]});


In [12]:
rows = session.execute("select (student_id, f_name, l_name, course, grade) from w04python.student")
for row in rows:
    print(f"student_id={row[0][0]}, f_name={row[0][1]}, l_name={row[0][2]}, grade={row[0][3]}, grade={row[0][4]}")


student_id=1004, f_name=Jim, l_name=Bean, grade=ISM6564, grade=65.0
student_id=1003, f_name=Betty, l_name=Johnson, grade=ISM6564, grade=78.0
student_id=1005, f_name=Frank, l_name=Gambali, grade=ISM6564, grade=92.0
student_id=1006, f_name=Judy, l_name=Garland, grade=ISM6562, grade=87.0
student_id=1002, f_name=Jane, l_name=Williams, grade=ISM6562, grade=89.0
student_id=1000, f_name=Tim, l_name=Smith, grade=ISM6562, grade=95.0
student_id=1001, f_name=John, l_name=Jones, grade=ISM6562, grade=97.0


In [13]:
rows = session.execute("select (l_name, f_name, grade) from w04python.student where grade > 80 ALLOW FILTERING")
for row in rows:
    print(f"{row[0][0]}, {row[0][1]}, {row[0][2]}")

Gambali, Frank, 92.0
Garland, Judy, 87.0
Williams, Jane, 89.0
Smith, Tim, 95.0
Jones, John, 97.0


## Challenge

Let's say we want to select all students with a grade higher than 90 but without requiring ALLOW FILTERING

Can you come up with a way to to do this?

Here are some resources to help you better understand cassandra ALLOW FILTERING and indexing (there are many other resources online):

> For more informaiton on ALLOW FILTERING see: https://www.datastax.com/blog/allow-filtering-explained